# CIFAR10 (precompute conv output)

https://github.com/keras-team/keras/blob/master/examples/cifar10_cnn.py

In [1]:
from theano.sandbox import cuda
cuda.use('gpu0')

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 950 (CNMeM is enabled with initial size: 90.0% of memory, cuDNN 5110)
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29



In [2]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, ZeroPadding2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import RMSprop, Adam
import bcolz

Using Theano backend.


In [3]:
batch_size = 32
num_classes = 10
epochs = 25

### The data, shuffled and split between train and test sets

In [4]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [5]:
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 3, 32, 32)
50000 train samples
10000 test samples


### Convert class vectors to binary class matrices

In [6]:
y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)

### Create model

In [7]:
model = Sequential()

In [8]:
model.add(ZeroPadding2D((1, 1), input_shape=x_train.shape[1:]))
model.add(Convolution2D(32, 3, 3, activation='relu'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))
model.add(Dropout(0.25))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [9]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 34, 34)     0           zeropadding2d_input_1[0][0]      
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 32, 32, 32)    896         zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 32, 16, 16)    0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 32, 16, 16)    0           maxpooling2d_1[0][0]             
___________________________________________________________________________________________

### initiate RMSprop optimizer

In [10]:
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

### Let's train the model using RMSprop

In [11]:
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [12]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [13]:
x_train /= 255
x_test /= 255

In [14]:
model.fit(x_train, y_train, batch_size=batch_size, nb_epoch=epochs, validation_data=(x_test, y_test), shuffle=True)

Train on 50000 samples, validate on 10000 samples
Epoch 1/25
50000/50000 [==============================] - 16s - loss: 1.8270 - acc: 0.3341 - val_loss: 1.5387 - val_acc: 0.4491
Epoch 2/25
50000/50000 [==============================] - 16s - loss: 1.4813 - acc: 0.4660 - val_loss: 1.3402 - val_acc: 0.5224
Epoch 3/25
50000/50000 [==============================] - 16s - loss: 1.3419 - acc: 0.5211 - val_loss: 1.2166 - val_acc: 0.5712
Epoch 4/25
50000/50000 [==============================] - 16s - loss: 1.2486 - acc: 0.5575 - val_loss: 1.1928 - val_acc: 0.5774
Epoch 5/25
50000/50000 [==============================] - 16s - loss: 1.1781 - acc: 0.5848 - val_loss: 1.1159 - val_acc: 0.6065
Epoch 6/25
50000/50000 [==============================] - 16s - loss: 1.1195 - acc: 0.6047 - val_loss: 1.0259 - val_acc: 0.6390
Epoch 7/25
50000/50000 [==============================] - 16s - loss: 1.0728 - acc: 0.6238 - val_loss: 0.9927 - val_acc: 0.6504
Epoch 8/25
50000/50000 [==============================

In [15]:
model.save_weights('results/cifar10.h5')

In [14]:
model.load_weights('results/cifar10.h5')

### Precompute convolution output

In [15]:
layer_type = Convolution2D
layers = model.layers
layer_idx = [index for index,layer in enumerate(layers) if type(layer) is layer_type][-1]

In [16]:
conv_layers = layers[:layer_idx+1]
fc_layers = layers[layer_idx+1:]

In [17]:
conv_model = Sequential(conv_layers)

In [18]:
conv_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 34, 34)     0           zeropadding2d_input_1[0][0]      
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 32, 32, 32)    896         zeropadding2d_1[0][0]            
                                                                   zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 32, 16, 16)    0           convolution2d_1[0][0]            
                                                                   convolution2d_1[1][0]            
___________________________________________________________________________________________

In [19]:
def save_array(fname, arr):
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()
    
def load_array(fname):
    return bcolz.open(fname)[:]    

In [20]:
conv_feat = conv_model.predict(x_train)

In [22]:
save_array('results/cifar10_conv_feat.dat', conv_feat)

In [21]:
conv_val_feat = conv_model.predict(x_test)

In [24]:
save_array('results/cifar10_conv_val_feat.dat', conv_val_feat)

In [20]:
conv_feat = load_array('results/cifar10_conv_feat.dat')
conv_val_feat = load_array('results/cifar10_conv_val_feat.dat')

In [22]:
conv_layers[-1].output_shape

(None, 64, 14, 14)

### FC only

In [23]:
def get_fc_model(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]), # first layer must have input_shape
        Dropout(p / 2),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(p),
        Dense(10, activation='softmax')        
    ]

#### Learning rate = 0.001, Dropout = 0.5

In [24]:
fc_model = Sequential(get_fc_model(0.5))
fc_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [25]:
fc_model.fit(conv_feat, y_train, batch_size=batch_size, nb_epoch=epochs, validation_data=(conv_val_feat, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/25
50000/50000 [==============================] - 4s - loss: 1.5022 - acc: 0.4946 - val_loss: 1.0740 - val_acc: 0.6517
Epoch 2/25
50000/50000 [==============================] - 4s - loss: 1.0557 - acc: 0.6376 - val_loss: 0.9270 - val_acc: 0.6849
Epoch 3/25
50000/50000 [==============================] - 4s - loss: 0.9393 - acc: 0.6773 - val_loss: 0.8634 - val_acc: 0.7070
Epoch 4/25
50000/50000 [==============================] - 4s - loss: 0.8781 - acc: 0.6947 - val_loss: 0.8378 - val_acc: 0.7128
Epoch 5/25
50000/50000 [==============================] - 4s - loss: 0.8413 - acc: 0.7088 - val_loss: 0.8128 - val_acc: 0.7212
Epoch 6/25
50000/50000 [==============================] - 4s - loss: 0.8150 - acc: 0.7170 - val_loss: 0.8026 - val_acc: 0.7223
Epoch 7/25
50000/50000 [==============================] - 4s - loss: 0.7958 - acc: 0.7248 - val_loss: 0.7965 - val_acc: 0.7254
Epoch 8/25
50000/50000 [==============================] - 4s 

## Add Batch normalization

In [26]:
def get_bn_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        BatchNormalization(axis=1), 
        Dropout(p / 2),
        Flatten(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(10, activation='softmax')        
    ]

#### Learning rate = 0.001, Dropout = 0.5

In [27]:
bn_model = Sequential(get_bn_layers(0.5))
bn_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [28]:
bn_model.fit(conv_feat, y_train, batch_size=batch_size, nb_epoch=epochs, validation_data=(conv_val_feat, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/25
50000/50000 [==============================] - 5s - loss: 1.4366 - acc: 0.5617 - val_loss: 0.8551 - val_acc: 0.7101
Epoch 2/25
50000/50000 [==============================] - 5s - loss: 1.0073 - acc: 0.6754 - val_loss: 0.7903 - val_acc: 0.7304
Epoch 3/25
50000/50000 [==============================] - 5s - loss: 0.8881 - acc: 0.7059 - val_loss: 0.7637 - val_acc: 0.7422
Epoch 4/25
50000/50000 [==============================] - 5s - loss: 0.8064 - acc: 0.7298 - val_loss: 0.7511 - val_acc: 0.7481
Epoch 5/25
50000/50000 [==============================] - 5s - loss: 0.7592 - acc: 0.7391 - val_loss: 0.7467 - val_acc: 0.7515
Epoch 6/25
50000/50000 [==============================] - 5s - loss: 0.7166 - acc: 0.7550 - val_loss: 0.7372 - val_acc: 0.7565
Epoch 7/25
50000/50000 [==============================] - 5s - loss: 0.6855 - acc: 0.7667 - val_loss: 0.7352 - val_acc: 0.7558
Epoch 8/25
50000/50000 [==============================] - 5s 

| | loss | acc | val_loss | val_acc |
|---|---|---|---|---|
| lr=0.001, dropout=0.5 | 0.6818 | 0.7733 | 0.7677 | 0.7479 |
| BN, lr=0.001, dropout=0.5 | 0.4740 | 0.8402 | 0.7214 | 0.7702 |